In [8]:
from pathlib import Path
import numpy as np
import pandas as pd

from mne.io.base import BaseRaw

ModuleNotFoundError: No module named 'mne'

In [1]:
mode = {'CR':'corneal reflection', 'P':'pupil'}

filter_type = {'0': 'filter off',
               '1': 'standard filter',
               '2': 'extra filter'}
# Leading A is for Arm
# next R is for Remote
# B is for Binocular/Monocular.
# M is for Monocular only
# Final R means illuminator on Right (legacy systems)
mount= {'MTABLER': 'Desktop, Stabilized Head, Monocular',
        'BTABLER': 'Desktop, Stabilized Head, Binocular/Monocular',
        'RTABLER': 'Desktop (Remote mode), Target Sticker, Monocular',
        'RBTABLER': 'Desktop (Remote mode), Target Sticker, Binocular/Monocular',
        'AMTABLER': 'Arm Mount, Stabilized Head, Monocular',
        'ABTABLER': 'Arm Mount Stabilized Head, Binocular/Monocular',
        'ARTABLER': 'Arm Mount (Remote mode), Target Sticker, Monocular',
        'ABRTABLE': 'Arm Mount (Remote mode), Target Sticker, Binocular/Monocular',
        'BTOWER': 'Binocular Tower Mount, Stabilized Head, Binocular/Monocular',
        'TOWER': 'Tower Mount, Stabilized Head, Monocular',
        'MPRIM': 'Primate Mount, Stabilized Head, Monocular',
        'BPRIM': 'Primate Mount, Stabilized Head, Binocular/Monocular',
        'MLRR': 'Long-Range Mount, Stabilized Head, Monocular, Camera Level',
        'BLRR': 'Long-Range Mount, Stabilized Head, Binocular/Monocular, Camera Angled'}

In [2]:
def _get_header(asc_fname):
    header_info = {}
    is_header = False
    for line in Path(asc_fname).open():
        if line.startswith('**'):
            is_header = True
        else:
            return header_info
        if is_header:
            hdr = line.lstrip('** ').split(':',maxsplit=1)
            if hdr and hdr[0].isupper():
                header_info[hdr[0]] = hdr[1].strip()

                
def _get_data_spec(asc_fname):
    data_spec = {}
    with Path(asc_fname).open() as file:
        is_data_spec = False
        for line in file:
            if line.isspace():
                continue
            if 'RECCFG' in line:
                is_data_spec = True
            if is_data_spec:
                if 'RECCFG' in line:
                    info = line.split('RECCFG')[1].split()
                    data_spec['tracking_mode'] = mode[info[0]]
                    data_spec['srate'] = int(info[1])
                    data_spec['sample_filter'] = filter_type[info[2]]
                    data_spec['analog_filter'] = filter_type[info[3]]
                    data_spec['eyes_tracked'] = info[4]
                elif 'ELCLCFG' in line:
                    info = line.split('ELCLCFG')[1].split()
                    data_spec['mount_config'] = mount[info[0]]
                elif 'GAZE_COORDS' in line:
                    line.find('L')
                    info = line.split('GAZE_COORDS')[1].split()
                    data_spec['pixel_resolution'] = {'top-left': {'x':float(info[0]),
                                                                  'y':float(info[1])},
                                                     'top-right':{'x':float(info[2]),
                                                                  'y':float(info[3])}}
                elif 'THRESHOLDS' in line:
                    pass
                else:
                    return data_spec

                
def _get_tracking_info(asc_fname):
    tracking_info = _get_data_spec(asc_fname)
    tracking_info['camera'] = _get_header(asc_fname)['CAMERA']
    return tracking_info


def _replace_missing_vals(line):
    '''return list where missing gaze data
       (indicated by '.') are replaced by np.nan.'''
    return [np.nan if i == '.' else i for i in line]

def _is_sys_msg(line):
    return any(['!V' in line,
               '!MODE' in line,
              ';' in line])

def _parse_recording_blocks(asc_fname):
    with Path(asc_fname).open() as file:
        samples = []
        events = {'START':[], 'END':[], 'SAMPLES':[], 'EVENTS':[],
                  'ESACC':[], 'EBLINK':[], 'EFIX':[],
                  'MSG':[], 'INPUT':[], 'BUTTON': []}      
        
        is_recording_block = False
        for line in file:
            if line.startswith('START'):
                is_recording_block = True
            if is_recording_block and not _is_sys_msg(line):
                line = _replace_missing_vals(line.split())
                if line[0].isdigit():  # Sample lines start with a number.
                    samples.append(line)
                elif line[0] in events.keys():
                        events[f'{line[0]}'].append(line[1:])
                if line[0] == 'END':
                    is_recording_block = False
        return samples, events

In [3]:
def read_raw_eyelink(asc_fname):
    return RawEyelink(asc_fname)
    
class RawEyelink():
    def __init__(self, asc_fname):
        self.asc_fname = Path(asc_fname)
        self.tracking_info = _get_tracking_info(self.asc_fname)
        self.session_info = _get_header(self.asc_fname)
        self._samples, self.events = _parse_recording_blocks(self.asc_fname)

In [6]:
raw = read_raw_eyelink('s04s07_AS_18Feb22.asc')

In [7]:
raw.tracking_info

{'tracking_mode': 'corneal reflection',
 'srate': 1000,
 'sample_filter': 'extra filter',
 'analog_filter': 'standard filter',
 'eyes_tracked': 'L',
 'mount_config': 'Arm Mount (Remote mode), Target Sticker, Binocular/Monocular',
 'pixel_resolution': {'top-left': {'x': 0.0, 'y': 0.0},
  'top-right': {'x': 1919.0, 'y': 1079.0}},
 'camera': 'Eyelink GL Version 1.2 Sensor=AG7'}